# Build an Agentic RAG Service

Setup an agent service that can interact with a tool service (containing RAG tools over annual reports).

In this notebook, we:
- Setup our indexes and query engine tools
- Define our multi-agent framework
  - A message queue.
  - An agentic orchestrator.
  - A tools service containing our query engine tools. This will act as a remote executor for tools
  - Define meta-tools for our agents. These will make calls to the tools service instead of executing directly
  - Our agent services. These wrap existing llama-index agents
  - Put all this into a local launcher, to simulate one task passing through the system at a time.

In [ ]:
import nest_asyncio

nest_asyncio.apply()

import os

os.environ["OPENAI_API_KEY"] = "sk-proj-..."

## Load Data

In [ ]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

--2024-06-23 18:22:44--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1880483 (1.8M) [application/octet-stream]
Saving to: ‘data/10k/uber_2021.pdf’

data/10k/uber_2021. 100%[===================>]   1.79M  7.33MB/s    in 0.2s    

2024-06-23 18:22:44 (7.33 MB/s) - ‘data/10k/uber_2021.pdf’ saved [1880483/1880483]

--2024-06-23 18:22:44--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connec

In [ ]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [ ]:
try:
    storage_context = StorageContext.from_defaults(persist_dir="./storage/lyft")
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(persist_dir="./storage/uber")
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

In [ ]:
if not index_loaded:
    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["./data/10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["./data/10k/uber_2021.pdf"]
    ).load_data()

    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs)
    uber_index = VectorStoreIndex.from_documents(uber_docs)

    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")

In [ ]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

In [ ]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

## Setup Agents

Now that we've defined the query tools, we can wrap these under a `ToolService`.

In [ ]:
from llama_agents import (
    AgentService,
    ToolService,
    LocalLauncher,
    MetaServiceTool,
    ControlPlaneServer,
    SimpleMessageQueue,
    AgentOrchestrator,
)

from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.llms.openai import OpenAI


# create our multi-agent framework components
message_queue = SimpleMessageQueue()
control_plane = ControlPlaneServer(
    message_queue=message_queue,
    orchestrator=AgentOrchestrator(llm=OpenAI(model="gpt-4o")),
)

# define Tool Service
tool_service = ToolService(
    message_queue=message_queue,
    tools=query_engine_tools,
    running=True,
    step_interval=0.5,
)

# define meta-tools here
meta_tools = [
    await MetaServiceTool.from_tool_service(
        t.metadata.name,
        message_queue=message_queue,
        tool_service=tool_service,
    )
    for t in query_engine_tools
]


# define Agent and agent service
worker1 = FunctionCallingAgentWorker.from_tools(
    meta_tools,
    llm=OpenAI(),
)
agent1 = worker1.as_agent()
agent_server_1 = AgentService(
    agent=agent1,
    message_queue=message_queue,
    description="Used to answer questions over Uber and Lyft 10K documents",
    service_name="uber_lyft_10k_analyst_agent",
)

## Launch agent 

With our services, orchestrator, control plane, and message queue defined, we can test our llama-agents network by passing in single messages, and observing the results.

This is an excellent way to test, iterate, and debug your llama-agents system.

In [ ]:
import logging

# change logging level to enable or disable more verbose logging
logging.getLogger("llama_agents").setLevel(logging.INFO)

In [ ]:
## Define Launcher
launcher = LocalLauncher(
    [agent_server_1, tool_service],
    control_plane,
    message_queue,
)

In [ ]:
# query_str = "What was Lyft's revenue growth in 2021?"
# gets stuck in a loop, should mostly be called once
query_str = "What are the risk factors for Uber?"
result = launcher.launch_single(query_str)

INFO:llama_agents.message_queues.simple - Consumer bba9b1a0-1967-4c66-b8a7-4792448b5645 has been registered.
INFO:llama_agents.message_queues.simple - Consumer e5c4a597-8879-43c7-9630-6fb04685d23d has been registered.
INFO:llama_agents.message_queues.simple - Consumer 7284e326-6fa8-4f4f-9679-5057f9ed7b8d has been registered.
INFO:llama_agents.message_queues.simple - Consumer 258e48f0-28e1-4f1f-a7d8-9487cf69b9bc has been registered.
INFO:llama_agents.services.agent - uber_lyft_10k_analyst_agent launch_local
INFO:llama_agents.message_queues.base - Publishing message 'control_plane' with action 'new_task'
INFO:llama_agents.message_queues.simple - Launching message queue locally
INFO:llama_agents.message_queues.base - Publishing message 'uber_lyft_10k_analyst_agent' with action 'new_task'
INFO:llama_agents.message_queues.simple - Successfully published message 'control_plane' to consumer.
INFO:llama_agents.message_queues.simple - Successfully published message 'uber_lyft_10k_analyst_agent'

In [ ]:
print(result)

The risk factors for Uber include potential harm to their reputation, business, financial condition, and operating results due to criminal, violent, inappropriate, or dangerous activities by platform users, inadequate background checks on Drivers and Couriers, incidents of criminal activity or misconduct by Drivers or consumers, liability for safety incidents, negative press coverage, data security breaches, challenges in introducing new products and features, regulatory complexities, and compliance issues related to TNC regulations and worker classification laws such as California's AB5.

Additionally, the impact of the COVID-19 pandemic and responsive measures, natural disasters, economic downturns, public health crises or political crises, general macroeconomic conditions, competition in the industries, unpredictability of results of operations, uncertainty regarding the growth of ridesharing and other markets, ability to attract and retain qualified drivers and riders, insurance co

In [ ]:
query_str = "What was Lyft's revenue growth in 2021?"
result = launcher.launch_single(query_str)

INFO:llama_agents.message_queues.simple - Consumer 779ca69b-6198-4a04-a845-1b242c95981b has been registered.
INFO:llama_agents.message_queues.simple - Consumer d1e48e8a-e743-4468-9a01-10883813349b has been registered.
INFO:llama_agents.message_queues.simple - Consumer ecb6fdf4-de98-4853-9c27-d26cefd64e1d has been registered.
INFO:llama_agents.message_queues.simple - Consumer 37412168-1062-4710-a5c8-6971572c0f13 has been registered.
INFO:llama_agents.services.agent - uber_lyft_10k_analyst_agent launch_local
INFO:llama_agents.message_queues.base - Publishing message 'control_plane' with action 'new_task'
INFO:llama_agents.message_queues.simple - Launching message queue locally
INFO:llama_agents.message_queues.base - Publishing message 'uber_lyft_10k_analyst_agent' with action 'new_task'
INFO:llama_agents.message_queues.simple - Successfully published message 'control_plane' to consumer.
INFO:llama_agents.message_queues.simple - Successfully published message 'uber_lyft_10k_analyst_agent'

In [ ]:
print(result)

Lyft's revenue grew by 36% in 2021 compared to the previous year.
